In [ ]:
# Importeer benodigde modules
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import ipywidgets as widgets
import shutil
import os

# ImageResize
Dit script is bedoeld om foto's te upscale, downscale en draaien. Deze functionaliteit is bij veel van de scripts in deze repository al ingebouwd. Alleen hier vind je de functionaliteit vrijstaand.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de foto's een nieuw formaat gaan krijgen.
InputFotos : str = r"pad/naar/jouw/folder"
# Bijvoorbeeld: r"pad/naar/jouw/folder"

# Output map waar de resized foto's worden opgeslagen.
OutputLocatie : str = r"pad/naar/jouw/output/folder"
# Bijvoorbeeld: r"pad/naar/jouw/output/folder"

# Bepaal of de hele foto horizontaal(True) of verticaal(False) is.
HeleFotoHorizontaal = True
# Bijvoorbeeld: True OF False

# Bepaal of de rotatie naar rechts(True) of naar links(False) is.
RotatieNaarRechts = False
# Bijvoorbeeld: True OF False

# Geef de nieuwe langste zijde van de hele foto in pixels.
HeleFotoLangsteZijde = 3200
# Bijvoorbeeld: 3200 OF 1080 OF 720 etc.

# Bepaal of je de mapstructuur op de outputlocatie wilt behouden.
BehoudMapStructuur: bool = True 
# Bijvoorbeeld: True OF False

---
## **Code**

In [ ]:
#maakt een output scherm aan
OutputScreen = widgets.Output()
display(OutputScreen)

#ondersteunde bestandstypen (Lowercase)
ImageTypes = [".jpg", ".png", "jpeg"]

# Zoekt naar alle mappen met afbeeldingen in de input map zodat de laadbalk kan laten zien hoeveel en welke mappen hij verwerkt
LijstFotos = set()
for root, dirs, files in os.walk(InputFotos):
    for File in files:
        FileName, FileType = os.path.splitext(File)
        if FileType.lower() in ImageTypes:
            LijstFotos.add(root)
LijstFotos = sorted(LijstFotos)

# Bepaal de rotatiehoek op basis van RotatieNaarRechts
angle = (lambda: 90 if RotatieNaarRechts else 270)()

# Verwijderd de output map als deze al bestaat en maak alle mappen aan die nodig zijn
shutil.rmtree(OutputLocatie)
for Dir in LijstFotos:
    Dir = Path(Dir)
    if BehoudMapStructuur:
        MapStructuur = os.path.relpath(Dir, InputFotos)
        ImageSavePath = os.path.join(os.getcwd(), OutputLocatie, MapStructuur)
    else:
        ImageSavePath = os.path.join(os.getcwd(), OutputLocatie)

    os.makedirs(ImageSavePath)

with OutputScreen:
    # Weergeeft een laadbalk voor het aantal mappen
    for Dir in tqdm(LijstFotos, desc=f"Foto's upscalen", leave=True):
        # Weergeeft een laadbalk voor het aantal foto's in een map
        ReletiveDir = os.path.relpath(Dir, InputFotos)
        for Images in tqdm(os.listdir(Dir), desc=f"{ReletiveDir}", leave=True):
            ImageName, ImageType = os.path.splitext(Images)
            # Checkt of de bestandstype van de foto ondersteund wordt zodat mappen niet alleen afbeeldingen mogen/hoeven te bevatten
            if ImageType.lower() in ImageTypes:
                with Image.open(os.path.join(Dir, Images)) as ImageView:
                    Width, Height = ImageView.size

                    # Checkt of de foto gedraaid moet worden
                    if Width > Height:
                        if not HeleFotoHorizontaal:
                            ImageView = ImageView.rotate(angle, expand=True)
                            Width, Height = ImageView.size
                    elif Height > Width:
                        if HeleFotoHorizontaal:
                            ImageView = ImageView.rotate(angle, expand=True)
                            Width, Height = ImageView.size

                    # Berekend de nieuwe hoogte van de foto zodat verhoudingen niet veranderd worden en upscaled/downscaled de foto
                    if Width>Height and Width != HeleFotoLangsteZijde:
                        NewHeight = int(HeleFotoLangsteZijde/Width*Height)

                        ImageView = ImageView.resize((HeleFotoLangsteZijde, NewHeight))
                        Width, Height = ImageView.size

                    elif Width <= Height and Height != HeleFotoLangsteZijde:
                        NewWidth = int(HeleFotoLangsteZijde/Height*Width)

                        ImageView = ImageView.resize((NewWidth, HeleFotoLangsteZijde))
                        Width, Height = ImageView.size

                    if BehoudMapStructuur:
                        # bepaalt de mapstructuur die overgenomen moet worden
                        MapStructuur = os.path.relpath(Dir, InputFotos)
                        ImageSavePath = os.path.join(os.getcwd(), OutputLocatie, MapStructuur)
                    else:
                        ImageSavePath = os.path.join(os.getcwd(), OutputLocatie)
                                            
                    ImageView.save(os.path.join(ImageSavePath, Images))